# Ising-Lenz Ergodicity: Magnetisation data generation

    
     (c) 2013, 2014, 2015, 2016, 2025 Süzen
     GPL v3 

This notebook generates Ising-Lenz 1D magnetisation data `ising1Dmagnetisation.rds` under `datasets`.

* Workhorse of the data generation is method `average_magnetisation_traj`. The RDS file is simply 
an object of list, each member is the output of this method, a named list. See return of this method.
Average magnetisations are reported there, 10 repeats
* Parameters are hardcoded in paralell method below.
     * Sizes at Metropolis (`transP=1`)  and Glauber (`transP=2`) Dynamics `N=512, 1024, 1536`.
     * Temperature ranges `seq(from = 0.5, by = 0.06, length.out = 16)`.
     * `J=1.0 H=1.0`.
     * MC steps `2e6`, writes out the config at `1e3` steps.

##  Utilities 

Here we load the library and utilities.

In [1]:
# Average Magnetisation
rm(list=ls()) 
require("isingLenzMC"); # Developed and tested on v0.2.5 and R v4.5.1
library(parallel)       # Core package

source("../src/power_utilities.R");

Loading required package: isingLenzMC



In [2]:
parallel_average_magnetisation <- function(size, dynamics = 1, num_cores = 8) {
    # Parallel runs
    ntemp <- 16
    max_mc_time <- 2e6
    delta_mc_time <- 1e3
    temperatures <- seq(from = 0.5, by = 0.06, length.out = ntemp)
    mc_mag <- function(beta) average_magnetisation_traj(size, beta, J = 1.0, H = 1.0, transP = dynamics, max_mc_time, delta_mc_time, nrepeat = 200)

    current_temp <- 1
    results <- vector("list", ntemp)
    cycle <- as.integer(ntemp/num_cores)
    temp_count <- 0

    for(i in 1:cycle) {
        fromTemp <- temp_count+1
        toTemp <- temp_count + num_cores
        par_result <- mclapply(temperatures[fromTemp:toTemp], mc_mag, mc.cores = num_cores)
        for (j in 1:num_cores) {
            results[[current_temp]] <- par_result[[j]]
            current_temp <- current_temp + 1
        }
        temp_count <- temp_count + num_cores
    }
    results
}

## Data generation

Here we run MC steps to get the dataset and store it under `../datasets`

In [3]:
# ~420 min on 8 core MacBook-M2
result <- parallel_average_magnetisation(512, dynamics = 1)
result <- c(result, parallel_average_magnetisation(512, dynamics = 2))
result <- c(result, parallel_average_magnetisation(1024, dynamics = 1))
result <- c(result, parallel_average_magnetisation(1024, dynamics = 2))
result <- c(result, parallel_average_magnetisation(1536, dynamics = 1))
result <- c(result, parallel_average_magnetisation(1536, dynamics = 2))


In [4]:
saveRDS(result, "../datasets/ising1Dmagnetisation.rds")